In [1]:
!pip install tensorflow


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

2023-03-13 12:58:24.173418: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 12:58:26.929468: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 12:58:26.929515: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 12:58:32.630686: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo


# Load and preprocess the source data


In [18]:


# Load and preprocess the source data
source_data = pd.read_csv('electronics.csv', delimiter=',')
source_data = (source_data - np.mean(source_data, axis=0)) / np.std(source_data, axis=0)


# Define the GAN model

In [ ]:
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(32, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(source_data.shape[1], activation='linear'))
    return model

In [ ]:


def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu', input_dim=source_data.shape[1]))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

latent_dim = 10
generator = build_generator(latent_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)


# Train the GAN model

In [ ]:
epochs = 1000
batch_size = 32

for epoch in range(epochs):
    # Generate random noise as input to the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    # Generate synthetic data using the generator
    synthetic_data = generator.predict(noise)
    # Concatenate the source data and synthetic data
    combined_data = np.concatenate([source_data, synthetic_data])
    # Create labels for the source and synthetic data
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    # Train the discriminator on the combined data
    discriminator_loss = discriminator.train_on_batch(combined_data, labels)
    # Generate new random noise as input to the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    # Create labels for the synthetic data
    misleading_labels = np.zeros((batch_size, 1))
    # Train the GAN on the random noise and misleading labels
    gan_loss = gan.train_on_batch(noise, misleading_labels)
    # Print the losses for each epoch
    print('Epoch %d: Discriminator loss: %f, GAN loss: %f' % (epoch, discriminator_loss, gan_loss))

In [ ]:
# Generate synthetic data using the generator
synthetic_data = generator.predict(np.random.normal(0, 1, size=(source_data.shape[0], latent_dim)))
# Save the synthetic data to a file
np.savetxt('synthetic_customer_purchase_history.csv', synthetic_data, delimiter=',')